### Fine-Tuning Similarity Model Notebook

This notebook fine-tunes a similarity model to predict the similarity between code snippets (whether AI-generated or not).

- **Optional**: The fine-tuned model has been uploaded to my Hugging Face hub (username: `wasabibish`), making this notebook optional to run.

In [4]:
import pandas as pd

from datasets import Dataset, DatasetDict

from sklearn.metrics import mean_squared_error, r2_score

from sentence_transformers import SentenceTransformer, losses, SentenceTransformerTrainer, util
from sentence_transformers.training_args import SentenceTransformerTrainingArguments as TrainingArguments
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

## Data Preparation

In [5]:
data = pd.read_csv('data.csv')

In [6]:
#create a training dataframe
dataset_training = dataset_df = pd.DataFrame(columns=["sentence1", "sentence2", "score"])

# the dataset containing a pair (humain/given code, llm generated code) and the plagiarism score
dataset_training['sentence1'] = data['human_content']
dataset_training['sentence2'] = data['llm_content']
dataset_training['score'] = data['plagiarism_score']

In [7]:
# create a dataset from the dataframe
dataset = Dataset.from_pandas(dataset_training)

In [8]:
# split the dataset to train and test
dataset_dict = dataset.train_test_split(test_size=0.2)
train_dataset = dataset_dict['train']
test_dataset = dataset_dict['test']

# Traininig

## Training configs

In [9]:
model_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
# load the model (sentence similarity model)
model = SentenceTransformer(model_name)

In [11]:
# define an evaluator for the model
evaluator = EmbeddingSimilarityEvaluator(test_dataset['sentence1'], test_dataset['sentence2'], test_dataset['score'])

In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    max_steps=100,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.2,
    save_steps=50,
    warmup_steps=150,
    evaluation_strategy="steps",
    eval_steps=20,
    report_to="none"
)

In [13]:
# define the loss that will be used to train the model
loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
# define the trainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    evaluator=evaluator,
    loss=loss
)

## Train

In [ ]:
# train the model
trainer.train()

In [31]:
finetuned_model = trainer.model

## Evaluation

In [ ]:
# evaluate the model on the test dataset
evaluator(finetuned_model)

# Inference

In [63]:
def populate_vector_db(test_dataset, finetuned_model):
    """
    Simulate a database of vectors for the test dataset

    Parameters:
    ----------
    test_dataset: pd.DataFrame
        The test dataset
    finetuned_model: SentenceTransformer
        The fine-tuned model

    Returns:
    -------
    vector_db: dict
        A dictionary containing the vectors and scores of the test dataset
    """

    # Create an empty dictionary to store vectors and scores
    vector_db = {}


    for i in range(len(test_dataset)):
        sentence1 = test_dataset['sentence1'][i]
        sentence2 = test_dataset['sentence2'][i]
        score = test_dataset['score'][i]
        
        # encode sentences (create the embeddings)
        vector1 = finetuned_model.encode(sentence1)
        vector2 = finetuned_model.encode(sentence2)

        # save them with the metadata
        vector_db[sentence1] = {
            'vector': vector1,
            'score': score
        }

        vector_db[sentence2] = {
            'vector': vector2,
            'score': score
        }

    return vector_db


In [64]:
vector_db = populate_vector_db(test_dataset, finetuned_model)

In [66]:
def calculate_weighted_score(sentence1_vector, vector_db,  top_n=3):

    """
    Calculate the weighted score of a given sentence based on the similarity scores 
    of the top_n most similar sentences in the vector_db

    Parameters:
    ----------
    sentence1_vector: np.array
        The vector of the sentence for which the weighted score should be calculated
    vector_db: dict
        A dictionary containing the vectors and scores of the test dataset
    top_n: int
        The number of most similar sentences to consider

    Returns:
    -------
    weighted_score: float
        The weighted score of the sentence
    """

    similarity_scores = {}

    # calculate similarity scores for each sentence in the vector_db
    for sentence, data in vector_db.items():
        vector = data['vector']
        score = data['score']
        similarity = util.pytorch_cos_sim(sentence1_vector, vector).item()
        similarity_scores[sentence] = {
            'similarity': similarity,
            'score': score
        }

    # sort sentences based on similarity scores
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda x: x[1]['similarity'], reverse=True)
    most_similar_sentences = sorted_similarity_scores[:top_n]

    # calculate the weighted score (similarity * real score)
    total_score = 0
    total_similarity = 0
    for sentence, data in most_similar_sentences:
        similarity = data['similarity']
        score = data['score']
        total_score += similarity * score
        total_similarity += similarity

    weighted_score = total_score / total_similarity if total_similarity > 0 else 0

    return weighted_score

In [68]:
def calculate_all_weighted_scores(test_dataset, vector_db, finetuned_model):
    """
    Calculate the weighted scores for all sentences in the test dataset

    Parameters:
    ----------
    test_dataset: pd.DataFrame
        The test dataset
    vector_db: dict
        A dictionary containing the vectors and scores of the test dataset
    finetuned_model: SentenceTransformer
        The fine-tuned model

    Returns:
    -------
    weighted_scores: list
        A list of weighted scores for all sentences in the test dataset
    """

    weighted_scores = []

    for i in range(len(test_dataset)):
        sentence1 = test_dataset['sentence1'][i]

        # embbed sentence
        sentence1_vector = finetuned_model.encode(sentence1)

        # ealculate weighted score
        weighted_score = calculate_weighted_score(sentence1_vector, vector_db)
        weighted_scores.append(weighted_score)

    return weighted_scores

In [70]:
predictions = calculate_all_weighted_scores(test_dataset, vector_db, finetuned_model)

In [78]:
def evaluate_performance(predictions, test_dataset):
    """
    Evaluate the performance of the model based on the predictions and the true scores

    Parameters:
    ----------
    predictions: list
        The predicted scores
    test_dataset: pd.DataFrame
        The test dataset

    Returns:
    -------
    mse: float
        The mean squared error
    r2: float
        The r2 score
    """
    true_scores = test_dataset['score']
    mse = mean_squared_error(true_scores, predictions)
    r2 = r2_score(true_scores, predictions)

    return mse, r2

In [ ]:
evaluate_performance(predictions, test_dataset)

In [ ]:
# push the model to huggingface
model_name = 'similarity-code-ai-generated'
finetuned_model.push_to_hub(model_name)